In [ ]:
import torch

device = (
    "cuda" if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available()
    else "cpu"
)

device

In [ ]:
import torch
import pandas as pd 

feature_indexes = [
    1,  # distance_from_last_transaction
    2,  # ratio_to_median_purchase_price
    4,  # used_chip
    5,  # used_pin_number
    6,  # online_order
]

label_indexes = [7]

# feature_columns = [
#     "distance_from_last_transaction",
#     "ratio_to_median_purchase_price",
#     "used_chip",
#     "used_pin_number",
#     "online_order",
# ]

# label_column = ["fraud"]

# train_df = pd.read_csv('data/train_sample.csv')
train_df = pd.read_csv('data/train.csv')
# labels_df = train_df.loc[:, label_column]
# train_df = train_df.loc[:, feature_columns]
labels_df = train_df.iloc[:, label_indexes]
train_df = train_df.iloc[:, feature_indexes]
train_df_tensor = torch.tensor(train_df.values, dtype=torch.float).to(device)
labels_df_tensor = torch.tensor(labels_df.values, dtype=torch.float).to(device)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch

# train_df = pd.read_csv('data/train_sample.csv')
train_df = pd.read_csv('data/train.csv')
# labels_df = train_df.loc[:, label_column]
# train_df = train_df.loc[:, feature_columns]
labels_df = train_df.iloc[:, label_indexes]
train_df = train_df.iloc[:, feature_indexes]

sk_scaler = StandardScaler()
sk_scaler.fit(train_df)
sk_scaler.mean_, sk_scaler.scale_

In [ ]:
import torch


# like scikit learn standard scaler
class TorchStandardScaler:
    def __init__(self):
        self.mean = None
        self.std = None

    def fit(self, tensor):
        self.mean = tensor.mean(dim=0, keepdim=False)
        self.std = tensor.std(dim=0, keepdim=False)

    def transform(self, tensor):
        return (tensor - self.mean) / self.std

    def fit_transform(self, tensor):
        self.fit(tensor)
        return self.transform(tensor)


train_df_tensor = torch.tensor(train_df.values, dtype=torch.float).to(device)
scaler = TorchStandardScaler()
scaler.fit(train_df_tensor)
scaler.mean, scaler.std

In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader


class CSVDataset(Dataset):
    def __init__(self, csv_file, transform=None, target_transform=None):
        self.feature_indexes = feature_indexes
        self.label_indexes = label_indexes
        # self.feature_columns = feature_columns
        # self.label_column = label_column 
        self.data = pd.read_csv(csv_file)
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        features = torch.tensor(self.data.iloc[idx, self.feature_indexes],
                                dtype=torch.float32).to(device)
        label = torch.tensor(self.data.iloc[idx, self.label_indexes],
                             dtype=torch.float32).to(device)
        # features = torch.tensor(self.data.loc[idx, self.feature_columns],
        #                         dtype=torch.float32).to(device)
        # label = torch.tensor(self.data.loc[idx, self.label_column],
        #                      dtype=torch.float32)
        if self.transform:
            features = self.transform(features)
        if self.target_transform:
            label = self.target_transform(label)

        return features, label



# training_data = CSVDataset('data/train_sample.csv', transform=scaler.transform)
# validation_data = CSVDataset('data/validate_sample.csv', transform=scaler.transform)
training_data = CSVDataset('data/train.csv', transform=scaler.transform)
validation_data = CSVDataset('data/validate.csv', transform=scaler.transform)

In [ ]:
from torch.utils.data import DataLoader

batch_size = 256
# batch_size = 8  # sample batch size

training_dataloader = DataLoader(training_data, batch_size=batch_size)
validation_dataloader = DataLoader(validation_data, batch_size=batch_size)

In [ ]:
import torch
from torch import nn

first_layer_num = len(training_data.feature_indexes)


class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(first_layer_num, 32),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(32, 32),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(32, 32),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(32, 1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits


model = NeuralNetwork().to(device)
model

In [ ]:
# Initialize the loss function
labels_df_tensor = torch.tensor(labels_df.values, dtype=torch.float)
positives = torch.sum(labels_df_tensor)
negatives = (len(labels_df_tensor) - torch.sum(labels_df_tensor))
pos_weight = torch.unsqueeze((negatives / positives), 0)
print(pos_weight)

# loss_fn = nn.BCEWithLogitsLoss().to(device)
loss_fn = nn.BCEWithLogitsLoss(pos_weight=pos_weight).to(device)
# loss_fn = nn.CrossEntropyLoss().to(device)
loss_fn

In [ ]:
learning_rate = 0.01

# optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate)


In [ ]:
def accuracy_fn(y_pred, y_actual):
    correct = torch.eq(torch.round(y_pred), y_actual).sum().item()
    acc = (correct / len(y_pred)) * 100
    return acc

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
    global print_stuff
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % round(size / batch_size / 10) == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += torch.eq(torch.round(pred), y).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

We initialize the loss function and optimizer, and pass it to
`train_loop` and `test_loop`. Feel free to increase the number of epochs
to track the model\'s improving performance.


In [ ]:
model.state_dict()

In [ ]:
# loss_fn = nn.CrossEntropyLoss()
# optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(training_dataloader, model, loss_fn, optimizer)
    test_loop(validation_dataloader, model, loss_fn)
print("Done!")

### Save Model

In [ ]:
import os

model_dir = "models/fraud/1/"
os.makedirs(model_dir, exist_ok=True) 
torch.save(model, f"{model_dir}/model.pth")

### Test Model

In [ ]:
def run_inference(test_data):
    model.eval()
    with torch.inference_mode():
        prediction = torch.round(model(test_data))

    if prediction.item() == 1:
        return "fraud"
    else:
        return "NOT fraud"

In [ ]:
# valid transaction
valid_tx = torch.tensor([[0.0, 1.0, 1.0, 1.0, 0.0]]).to(device)
prediction = run_inference(valid_tx)
print(f"The model thinks the valid transaction is {prediction}")

In [ ]:
# fraudulent use case
fraud_tx = torch.tensor([[100, 1.2, 0.0, 0.0, 1.0]]).to(device)
prediction = run_inference(fraud_tx)
print(f"The model thinks the valid transaction is {prediction}")

In [ ]:
# test_df = pd.read_csv('data/test_sample.csv', )
test_df = pd.read_csv('data/test.csv', )
test_labels_df = test_df.iloc[:, label_indexes]
test_data_df = test_df.iloc[:, feature_indexes]
test_data_df_tensor = torch.tensor(test_data_df.values, dtype=torch.float).to(device)
test_labels_df_tensor = torch.tensor(test_labels_df.values, dtype=torch.float).to(device)

In [ ]:
model.eval()
with torch.inference_mode():
    y_pred = model(test_data_df_tensor)
    acc = accuracy_fn(y_pred, test_labels_df_tensor)

print(acc)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from matplotlib import pyplot as plt

print(f"Accuracy: {acc}")

c_matrix = confusion_matrix(torch.Tensor.cpu(test_labels_df_tensor),
                            torch.round(torch.Tensor.cpu(y_pred)))
ConfusionMatrixDisplay(c_matrix).plot()
